In [1]:
cd ../

/Users/tombutler/Documents/ai-hackathon


In [7]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score

from data.processed.pre_process_titanic import pre_process_df

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [3]:
TRAIN_DATA_PATH = "data/raw/train.csv"
TEST_DATA_PATH = "data/raw/test.csv"
FAKE_DATA_PATH = "data/fake/tabgan_data.csv"
NUM_COLUMNS = ["Fare", "Age"]
CAT_COLUMNS = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]
LABEL = "Survived"
COLUMNS = NUM_COLUMNS + CAT_COLUMNS

### 1. Load in the data and prepare it

In [4]:
# Load the Titanic dataset
train = pre_process_df(TRAIN_DATA_PATH)[COLUMNS]
target = pd.read_csv(TRAIN_DATA_PATH)[[LABEL]]
test = pre_process_df(TEST_DATA_PATH)[COLUMNS]

fake = pd.read_csv(FAKE_DATA_PATH)

In [5]:
# Impute missing values
imputer = SimpleImputer(strategy="median")
train = pd.DataFrame(imputer.fit_transform(train), columns=COLUMNS)
test = pd.DataFrame(imputer.fit_transform(test), columns=COLUMNS)

### 2. Extract a test set, and set up the real and fake features and targets

In [6]:
X = train[COLUMNS].copy()
y = target.copy()

In [8]:
X_train_real, X_test, y_train_real, y_test = train_test_split(X, y, stratify=y)

In [9]:
X_train_fake = fake[COLUMNS].copy()
y_train_fake = fake[LABEL].copy()

### 3. Train the models

In [14]:
model_real = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model_real.fit(X_train_real, y_train_real)
predictions_real = model_real.predict(X_test)

/var/folders/qm/y349637s1v91yky4ss3rlmn80000gn/T/ipykernel_59737/3585395417.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_real.fit(X_train_real, y_train_real)


In [15]:
model_fake = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model_fake.fit(X_train_fake, y_train_fake)
predictions_fake = model_fake.predict(X_test)

### 4. Compare the models

In [13]:
def get_metrics(y_test, predictions):
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)

    return {"accuracy": accuracy, "precision": precision, "recall": recall}

In [16]:
metrics_real = get_metrics(y_test, predictions_real)
metrics_fake = get_metrics(y_test, predictions_fake)

/Users/tombutler/miniforge3/envs/jupyter-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
metrics_real

{'accuracy': 0.820627802690583,
 'precision': 0.896551724137931,
 'recall': 0.6046511627906976}

In [18]:
metrics_fake

{'accuracy': 0.6143497757847534, 'precision': 0.0, 'recall': 0.0}

In [9]:
print(f"Accuracy: {accuracy:,.2f}")
print(f"Precision: {precision:,.2f}")
print(f"Recall: {recall:,.2f}")

Accuracy: 0.72
Precision: 0.70
Recall: 0.49
